## Model Training

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df = df.drop(labels=['id'],axis=1)

In [4]:
## Independent and Dependent features

X = df.drop(labels=['price'],axis=1)
y = df[['price']]

In [5]:
# Define which columns should be ordinal encoded and which should be scaled

categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [6]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']



In [7]:
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import StandardScaler ## Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder ## Feature Engineering (Ordinal Encoding)

## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [8]:
## Numerical Pipeline

num_pipeline = Pipeline(
                steps = [
                ('imputer',SimpleImputer(strategy='median')),
                ('scaler',StandardScaler())                
                ]
            )

## Categorical Piperline

cat_pipeline = Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinal_encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
                ('scaler',StandardScaler())
                ]
            )

preprocessor = ColumnTransformer(
                [
                ('num_pipeline',num_pipeline,numerical_cols),
                ('cat_pipeline',cat_pipeline,categorical_cols)
                ]
            )


In [9]:
## Train test Split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=30)


In [11]:
preprocessor.fit_transform(x_train)

array([[-0.54220555,  2.11279474, -1.68414474, ..., -2.1467829 ,
         0.93331879,  0.68600707],
       [-0.15318745,  0.44710281, -1.16300271, ...,  0.87370984,
         0.31760568, -1.31372343],
       [ 1.57578188, -2.14397354,  0.92156539, ..., -1.13995199,
         0.31760568, -0.6471466 ],
       ...,
       [ 0.45195182,  1.5575641 , -0.64186069, ..., -2.1467829 ,
        -0.29810744, -0.6471466 ],
       [ 0.66807298, -1.77381977,  1.44270741, ...,  0.87370984,
        -0.29810744,  0.68600707],
       [ 0.25744277,  0.81725657, -0.12071866, ...,  0.87370984,
        -0.29810744, -0.6471466 ]])